# Install libs

In [0]:
%pip install stravalib

# Setup environment variables

In [0]:
import os

os.environ["STRAVA_CLIENT_ID"] = dbutils.secrets.get(scope = "strava", key = "client_id")
os.environ["STRAVA_CLIENT_SECRET"] = dbutils.secrets.get(scope = "strava", key = "client_secret")

# Instantiate a client object

In [0]:
import time
from stravalib import Client

# --- Strava client ---
client = Client(
    access_token=dbutils.secrets.get(scope = "strava", key = "access_token"),
    refresh_token=dbutils.secrets.get(scope = "strava", key = "refresh_token"),
    token_expires=int(dbutils.secrets.get(scope = "strava", key = "expires_at")),
)

# --- API call ---
athlete = client.get_athlete()
print(f"Hi, {athlete.firstname} {athlete.lastname} Welcome to stravalib!")

# Save data to catalog

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType

activity_schema = StructType([
    StructField("id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("start_date_local", TimestampType(), True),
    StructField("average_speed_mps", DoubleType(), True),
    StructField("distance_m", DoubleType(), True),
    StructField("total_elevation_gain_m", DoubleType(), True),
    StructField("sport_type", StringType(), True),
    StructField("pr_count", IntegerType(), True),
    StructField("kudos_count", IntegerType(), True)
])


activities_list = []
for activity in client.get_activities():
    activities_list.append({
        "id": str(activity.id),
        "name": activity.name,
        "start_date_local": activity.start_date_local,
        "average_speed_mps": float(activity.average_speed) if activity.average_speed else None,
        "distance_m": float(activity.distance) if activity.distance else None,
        "total_elevation_gain_m": float(activity.total_elevation_gain) if activity.total_elevation_gain else None,
        "sport_type": activity.sport_type,
        "pr_count": int(activity.pr_count) if activity.pr_count is not None else None,
        "kudos_count": int(activity.kudos_count) if activity.kudos_count is not None else None
    })

df = spark.createDataFrame(activities_list, schema=activity_schema)
df.write.format("delta").mode("overwrite").saveAsTable("workspace.default.bronze_strava_activities")